# If you are reading these lines, then you probably want to do serious stuff with PyMoosh.
Here are shown "advanced" new PyMoosh features, with examples. First you need to install ```Refractiveindex``` library.

```
pip python install --upgrade refractiveindex
```

# SPECIAL MATERIALS :
If you want to use other materials references (Refractiveindex.info, your own experimental data, etc) or anisotropic material, then you should look over there.

## (1/3) Refractiveindex 
To create a material from the Refractiveindex.info database, you can use the following lines. You can find the path name by hovering your cursor on the link in the Data section (see Refractiveindex git).

In [1]:
from PyMoosh.materials import Material

shelf, book, page = 'main', 'Ag', 'Rakic-BB'
path = [shelf, book, page]

special_material = Material(path, "RII")

/home/stagiaire1/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


```RII``` keyword will bring information from the Refractiveindex.info database.
Warning ! Each page has their own wavelength domain, and working with a wavelength outside this domain will raise an error.
Try this to see the helpful error: 

In [ ]:
wavelength = 100 # nm

epsilon = special_material.get_permittivity(wavelength) 

# Exception : Wavelength 0.1 um is out of bounds. Correct range: (0.24797 um, 12.398 um)

You can add this material in your structure and use as usual other PyMoosh features without any changes.

In [2]:
import numpy
from PyMoosh.core import Structure

wavelength = 600 # nm
material_list = [1.0, special_material]
thickness = [1, 2]*wavelength

interface = Structure(material_list, [0, 1], thickness)

mat_1 = interface.materials[0]
mat_2 = interface.materials[1]

refractive_index_mat_1 = numpy.sqrt(mat_1.get_permittivity(wavelength))
refractive_index_mat_2 = numpy.sqrt(mat_2.get_permittivity(wavelength))

print(refractive_index_mat_1)
print(refractive_index_mat_2)

List of materials:
Simple, non dispersive: epsilon= (1+0j)
Object: Material
(1+0j)
(0.1519147989734816+3.5645603079555173j)


## (2/3) Anisotropic Material

To use anisotropic material from Refractiveindex.info, you can use the following lines. To select your birefringent material, first find the corresponding path as indicated in part one. Then use the ```ANI``` argument to make the code understand you are not looking for one page, but all pages corresponding to this material (There one page for each main refractive index).

In [3]:
from PyMoosh.materials import Material

shelf, book, page = 'main', 'As2S3', 'Slavich-alpha'
path = [shelf, book, page]

birefringent_material = Material(path, "ANI")

In this exemple we have a bi-axis material. In fact, one can replace ```-alpha``` by ```'-beta'``` or ```'-gamma'``` without consequences, or even not precise the suffix at all. You will only raise an error if the material is not known to be birefringent. All of this is also true for uni-axis material, with ```-o``` (resp. ```-e```) for ordinary (resp. extraordinary) suffix.

As seen in previous section, you can add this material in your structure as usual. However, this will trigger the ```Structure``` constructor to use appropriate functions for anisotropic problems. To acces the permittivity of an anisotropic material, you can use the ```get_permittivity_ani(incident_angle, precession, nutation, spin)``` method, which takes four more arguments than previous method:

- Incident angle of the light beam, in the incident plane. It is an angle between 0° and 90°.  

- The euler's angles, precession, nutation and spin, indicating the change of basis between the incident plane (xOz) and the main basis of the material.

By default, the main basis and the incident plane basis are the same.

In [4]:
import numpy
from PyMoosh.core import Structure

wavelength = 600 # nm
material_list = [1.0, birefringent_material]
thickness = [1, 2]*wavelength

interface = Structure(material_list, [0, 1], thickness)

mat_1 = interface.materials[0]
mat_2 = interface.materials[1]

refractive_index_mat_1 = numpy.sqrt(mat_1.get_permittivity(wavelength))

precession, nutation, spin = 0, 0, 0
refractive_index_mat_2 = numpy.sqrt(mat_2.get_permittivity_ani(wavelength, 45, precession, nutation, spin))

print(refractive_index_mat_1)
print(refractive_index_mat_2)

List of materials:
Simple, non dispersive: epsilon= (1+0j)
Object: Material
(1+0j)
[1.68671574 1.5896474 ]


The same error about the wavelength domain can occur, as seen before.

For the time being, there is no coefficient functions for this kind of material.

## (3/3) Experimental Data Material

Something in the way